In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use('ggplot')
import pandas as pd
import numpy as np
from PIL import Image
pd.set_option('display.max_columns', 500)
import sys, os
from torch.utils.data import Dataset, DataLoader
import torch, torchvision
from torch.autograd import Function
from torch.autograd import Variable
from torchvision import transforms
from torchvision import models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, roc_curve, auc
from itertools import cycle
from itertools import chain
# from sklearn.linear_model import LogisticRegression
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
from sklearn.metrics import classification_report
from imblearn.metrics import specificity_score
from dalib.modules import WarmStartGradientReverseLayer
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import pandas as pd
from derm7pt.dataset import Derm7PtDataset, Derm7PtDatasetGroupInfrequent
from derm7pt.vis import plot_confusion
from derm7pt.kerasutils import deep_features
dir_release = '../data/release_v0'
dir_meta = os.path.join(dir_release, 'meta')
dir_images = os.path.join(dir_release, 'images')
meta_df = pd.read_csv(os.path.join(dir_meta, 'meta.csv'))
train_indexes = list(pd.read_csv(os.path.join(dir_meta, 'train_indexes.csv'))['indexes'])
valid_indexes = list(pd.read_csv(os.path.join(dir_meta, 'valid_indexes.csv'))['indexes'])
test_indexes = list(pd.read_csv(os.path.join(dir_meta, 'test_indexes.csv'))['indexes'])

2023-08-30 11:10:07.369105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-30 11:10:07.965191: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# The full dataset before any grouping of the labels.
derm_data = Derm7PtDataset(dir_images=dir_images, 
                        metadata_df=meta_df.copy(), # Copy as is modified.
                        train_indexes=train_indexes, valid_indexes=valid_indexes, 
                        test_indexes=test_indexes)

# The dataset after grouping infrequent labels.
derm_data_group = Derm7PtDatasetGroupInfrequent(dir_images=dir_images, 
                                             metadata_df=meta_df.copy(), # Copy as is modified.
                                             train_indexes=train_indexes, 
                                             valid_indexes=valid_indexes, 
                                             test_indexes=test_indexes)

In [4]:
derm_data.dataset_stats()

Number of cases: 1011
Number of cases to train: 413
Number of cases to validate: 203
Number of cases to test: 395


In [5]:
img_path = '../data/release_v0/images/'
BCC = ['basal cell carcinoma'] 
NEV = ['blue nevus', 'clark nevus', 'combined nevus', 'congenital nevus', 'dermal nevus', 'recurrent nevus', 'reed or spitz nevus']
MEL = ['melanoma', 'melanoma (in situ)', 'melanoma (less than 0.76 mm)', 'melanoma (0.76 to 1.5 mm)', 'melanoma (more than 1.5 mm)', 'melanoma metastasis' ]
MISC = ['dermatofibroma', 'lentigo', 'melanosis', 'miscellaneous', 'vascular lesion']
SK = ['seborrheic keratosis']
PN = {'absent':0, 'typical':1, 'atypical':2}
STR = {'absent':0, 'regular':1, 'irregular':2}
PIG = {'absent':0, 'diffuse regular':1, 'localized regular':1, 'diffuse irregular':2, 'localized irregular':2}
RS = {'absent':0, 'blue areas':1, 'white areas':1, 'combinations':1}
DaG = {'absent':0, 'regular':1, 'irregular':2}
BWV = {'absent':0, 'present':1}
VS = {'absent':0, 'arborizing':1, 'comma':1, 'hairpin':1, 'within regression':1, 'wreath':1, 'dotted':2, 'linear irregular':2}
def get_diag_label(diag):
    if diag in BCC:
        label = 0
    elif diag in NEV:
        label = 1
    elif diag in MEL:
        label = 2
    elif diag in MISC:
        label = 3
    elif diag in SK:
        label = 4
    if label == None:
        print("Error!")
    else:
        return label
def get_7point_label(point_criteria):
    label0 = PN[point_criteria[0]]
    label1 = STR[point_criteria[1]]
    label2 = PIG[point_criteria[2]]
    label3 = RS[point_criteria[3]]
    label4 = DaG[point_criteria[4]]
    label5 = BWV[point_criteria[5]]
    label6 = VS[point_criteria[6]]
    return [label0, label1, label2, label3, label4, label5, label6]
clinic_train = []
clinic_validate = []
clinic_test = []
dermoscopic_train = []
dermoscopic_validate = []
dermoscopic_test = []
label_train_diag = []
label_validate_diag = []
label_test_diag = []
label_train_crit = []
label_validate_crit = []
label_test_crit = []
for index, row in meta_df.iterrows():
    c_img = row[15]
    d_img = row[16]
    diag = row[1]
    p_n = row[3]
    s_t_r = row[4]
    p_i_g = row[5]
    r_s = row[6]
    d_a_g = row[7]
    b_w_v = row[8]
    v_s = row[9]
    point_criteria = [p_n, s_t_r, p_i_g, r_s, d_a_g, b_w_v, v_s]
    if d_img == 'FCl/Fcl068.jpg':
        d_img = 'FCL/Fcl068.jpg'

    if index in train_indexes:        
        clinic_train.append(img_path + c_img)
        dermoscopic_train.append(img_path + d_img)
        label_train_diag.append(get_diag_label(diag))
        label_train_crit.append(get_7point_label(point_criteria))
    elif index in valid_indexes:
        clinic_validate.append(img_path + c_img)
        dermoscopic_validate.append(img_path + d_img)
        label_validate_diag.append(get_diag_label(diag))
        label_validate_crit.append(get_7point_label(point_criteria))
    elif index in test_indexes:
        clinic_test.append(img_path + c_img)
        dermoscopic_test.append(img_path + d_img)
        label_test_diag.append(get_diag_label(diag))
        label_test_crit.append(get_7point_label(point_criteria))
    else:
        print("There is an error need to be fixed!")


In [6]:
print(len(clinic_train), len(clinic_validate), len(clinic_test))
# print(clinic_train)

413 203 395


In [7]:
class MyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        img_c = Image.open(self.data.c_path[idx])
        img_c = img_c.convert("RGB")
        img_c = self.transform(img_c)
        img_d = Image.open(self.data.d_path[idx])
        img_d = img_d.convert("RGB")
        img_d = self.transform(img_d)
        
        label_diag_i = np.array(self.data.lab_diag[idx])
        
        label_crit_i = np.array(self.data.lab_crit[idx])
        
        return img_c, img_d, label_diag_i, label_crit_i

In [8]:
train_transforms = transforms.Compose([transforms.Resize([299, 299]),
                                       # transforms.Pad(padding=10, fill=(255, 176, 145)),
                                       transforms.RandomCrop([299, 299], padding=20, padding_mode='edge'),
                                       transforms.RandomHorizontalFlip(p=0.5),
                                       transforms.RandomRotation([-45, 45]),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.5 , 0.5 , 0.5), (0.5 , 0.5 , 0.5))])
test_transforms = transforms.Compose([transforms.Resize([299, 299]),
    transforms.ToTensor(),
    transforms.Normalize((0.5 , 0.5 , 0.5), (0.5 , 0.5 , 0.5))
])
image_transforms = {'train':train_transforms, 'test':test_transforms}

train = list(zip(clinic_train, dermoscopic_train, label_train_diag, label_train_crit))
train_df = pd.DataFrame(train, columns=['c_path','d_path','lab_diag', 'lab_crit'])
train_dataset = MyDataset(train_df, transform=image_transforms['train'])

validate = list(zip(clinic_validate, dermoscopic_validate, label_validate_diag, label_validate_crit))
validate_df = pd.DataFrame(validate, columns=['c_path','d_path','lab_diag', 'lab_crit'])
validate_dataset = MyDataset(validate_df, transform=image_transforms['test'])

test = list(zip(clinic_test, dermoscopic_test, label_test_diag, label_test_crit))
test_df = pd.DataFrame(test, columns=['c_path','d_path','lab_diag', 'lab_crit'])
test_dataset = MyDataset(test_df, transform=image_transforms['test'])

In [9]:
train_df

,c_path,d_path,lab_diag,lab_crit
0,../data/release_v0/images/NEL/NEL025.JPG,../data/release_v0/images/NEL/Nel026.jpg,0,"[0, 0, 0, 0, 0, 0, 1]"
1,../data/release_v0/images/NEL/Nel032.jpg,../data/release_v0/images/NEL/Nel033.jpg,0,"[0, 0, 0, 0, 2, 0, 1]"
2,../data/release_v0/images/NEL/NEL036.JPG,../data/release_v0/images/NEL/Nel037.jpg,0,"[0, 0, 2, 0, 0, 0, 0]"
3,../data/release_v0/images/NEL/Nel084.jpg,../data/release_v0/images/NEL/Nel085.jpg,0,"[0, 0, 2, 0, 2, 1, 0]"
4,../data/release_v0/images/NEL/NEL088.JPG,../data/release_v0/images/NEL/Nel089.jpg,0,"[0, 0, 2, 0, 2, 0, 1]"
...,...,...,...,...
408,../data/release_v0/images/NEL/Nel067.jpg,../data/release_v0/images/NEL/Nel066.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"
409,../data/release_v0/images/NEL/Nel069.jpg,../data/release_v0/images/NEL/Nel068.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"
410,../data/release_v0/images/NEL/Nel070.jpg,../data/release_v0/images/NEL/Nel071.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"
411,../data/release_v0/images/Ggl/Ggl011.jpg,../data/release_v0/images/Ggl/Ggl012.jpg,3,"[0, 0, 0, 0, 0, 0, 0]"


In [10]:
def cal_auc(pre, true, show = False):
    auc_all = {}
    
    plt.rc('font',family='Times New Roman')
    font1 = {'family': 'Times New Roman', 'weight': 'normal', 'size': 10,}
    for key in pre.keys():
        n_classes = np.array(pre[key]).shape[-1]
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        one_hot = torch.zeros(len(true[key]), n_classes).scatter_(1, torch.tensor(np.array(true[key]).reshape(len(np.array(true[key])), 1)), 1)
        for i in range(n_classes):
            # fpr[i], tpr[i], _ = roc_curve(one_hot[:, i], np.array(pre[key])[:, i])
            fpr[i], tpr[i], _ = roc_curve(one_hot[:, i], np.array(nn.Softmax(dim=1)(torch.Tensor(pre[key])))[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])
        fpr["micro"], tpr["micro"], _ = roc_curve(np.array(one_hot).ravel(), np.array(nn.Softmax(dim=1)(torch.Tensor(pre[key]))).ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

        all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
        # Then interpolate all ROC curves at this points
        mean_tpr = np.zeros_like(all_fpr)
        for i in range(n_classes):
            mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])
        # Finally average it and compute AUC
        mean_tpr /= n_classes
        fpr["macro"] = all_fpr
        tpr["macro"] = mean_tpr
        roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
        
        if show == True and key =='diag':
            # Plot all ROC curves
            color_list = ['aqua', 'darkorange', 'cornflowerblue', 'green', 'brown']
            lw=2
            plt.figure()

            colors = cycle(color_list[0:n_classes])
            
            name_diag=['BCC','NEV','MEL','MISC','SK']
            
            for i, color in zip(range(n_classes), colors):
                plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                         label='ROC curve of class {0} (area = {1:0.1f})'
                         ''.format(name_diag[i], roc_auc[i] * 100))

            plt.plot([0, 1], [0, 1], 'k--', lw=lw)
            plt.xlim([0.0, 1.0])
            plt.ylim([0.0, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            # plt.title('Some extension of Receiver operating characteristic to multi-class')
            plt.legend(loc="lower right", prop=font1)
            plt.savefig('./visualization/ROC.pdf')
            plt.show()
        auc_all[key] = roc_auc
    return auc_all

def cal_con_matrix(pre, true):
    acc_all = {}
    con_all = {}
    tp_fp = {}
    for key in pre.keys():
        acc = accuracy_score(np.array(true[key]), np.argmax(np.array(pre[key]), axis=-1))
        con = confusion_matrix(np.array(true[key]), np.argmax(np.array(pre[key]), axis=-1))
        acc_all[key] = acc
        con_all[key] = con
    return acc_all, con_all

def metric(pre, true, show = False):
    auc_all = cal_auc(pre=pre, true=true, show = show)
    acc_all, con_all = cal_con_matrix(pre=pre, true=true)
    return auc_all, acc_all, con_all

In [11]:
device = torch.device("cuda:0")# ("cuda:0")
class CNN(nn.Module): 
    def __init__(self, model):
        super(CNN, self).__init__()
        self.resnet_layer = nn.Sequential(*list(model.children())[:-2])
        
    def forward(self, x):
        
        x = self.resnet_layer(x)
        return x
class Concate(nn.Module): 
    def __init__(self):
        super(Concate, self).__init__()
        self.hidden_size = 512 # 512
        
        self.avp_pooling = nn.AdaptiveAvgPool2d((1, 1)) # AdaptiveAvgPool2d
        self.linear_layer1 = nn.Linear(2048 * 2, self.hidden_size) # reduce the dimensional
        
        # attention computation using SEblock
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(2048, 2048 // 2, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(2048 // 2, 2048, bias=False),
            nn.Sigmoid()
        )
        self.att_relu = nn.ReLU()
        
        # define classifiers
        self.out_diag = nn.Linear(self.hidden_size, 5)
        self.out_crit_pn = nn.Linear(self.hidden_size, 3) # p_n 3, s_t_r 3, p_i_g 3, r_s 2, d_a_g 3, b_w_v 2, v_s 3
        self.out_crit_str = nn.Linear(self.hidden_size, 3)
        self.out_crit_pig = nn.Linear(self.hidden_size, 3)
        self.out_crit_rs = nn.Linear(self.hidden_size, 2)
        self.out_crit_dag = nn.Linear(self.hidden_size, 3)
        self.out_crit_bwv = nn.Linear(self.hidden_size, 2)
        self.out_crit_vs = nn.Linear(self.hidden_size, 3)
        self.dropout = nn.Dropout(p=0.4)
        
    def forward(self, x_c, x_d):
        b, c, _, _ = x_c.size()
        img_size = torch.rand(x_c.size()[0], 3, x_c.size()[2], x_c.size()[3])
        # SE feature compute
        x_att_c = self.avg_pool(x_c).view(b, c)
        x_att_c = self.fc(x_att_c).view(b, c, 1, 1)
        x_att_feature_c = x_c * x_att_c.expand_as(x_c)
        
        x_att_d = self.avg_pool(x_d).view(b, c)
        x_att_d = self.fc(x_att_d).view(b, c, 1, 1)
        x_att_feature_d = x_d * x_att_d.expand_as(x_d)
        
        x_concat = torch.cat((x_c, x_d), dim=1)# x_att_feature_c, x_att_feature_d
         
        x_att_mask_c = torch.sum(x_c, dim=1) # x_att_feature_c
        x_att_mask_d = torch.sum(x_d, dim=1)
        # x_att_mask_c = x_att_c
        # x_att_mask_d = x_att_d
        x_att_mask_c = (x_att_mask_c - x_att_mask_c.min())/(x_att_mask_c.max() - x_att_mask_c.min()) #  (x - X_min) / (X_max - X_min)
        x_att_mask_d = (x_att_mask_d - x_att_mask_d.min())/(x_att_mask_d.max() - x_att_mask_d.min())
        # x_att_mask_c = torch.sum(x_att_feature_c, dim=1) / torch.sum(x_att_feature_c, dim=1).max()
        # x_att_mask_d = torch.sum(x_att_feature_d, dim=1) / torch.sum(x_att_feature_d, dim=1).max()
        x_att_mask_c = nn.functional.interpolate(x_att_mask_c.view(x_c.size()[0], 1, x_c.size()[2], x_c.size()[3]), size=(299, 299), mode='bilinear', align_corners=False) # bicubic, align_corners=False
        x_att_mask_d = nn.functional.interpolate(x_att_mask_d.view(x_d.size()[0], 1, x_d.size()[2], x_d.size()[3]), size=(299, 299), mode='bilinear', align_corners=False) # bicubic, align_corners=False
        
        # flatten feature vectors
        x = self.avp_pooling(x_concat).view(x_c.size()[0], -1)
        x = torch.relu(self.linear_layer1(x))
        x = self.dropout(x)
        # classifiers
        x_diag = self.out_diag(x)
        x_crit_pn = self.out_crit_pn(x)
        x_crit_str = self.out_crit_str(x)
        x_crit_pig = self.out_crit_pig(x)
        x_crit_rs = self.out_crit_rs(x)
        x_crit_dag = self.out_crit_dag(x)
        x_crit_bwv = self.out_crit_bwv(x)
        x_crit_vs = self.out_crit_vs(x)
        return x_diag, x_crit_pn, x_crit_str, x_crit_pig, x_crit_rs, x_crit_dag, x_crit_bwv, x_crit_vs, x_att_mask_c, x_att_mask_d, x_concat
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.hidden_size = 256
        self.grl = WarmStartGradientReverseLayer(alpha=1., lo=0., hi=1., max_iters=100, auto_step=True) 
        self.hidden_layer = nn.Linear(2048, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, 2)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.dropout = nn.Dropout(p=0.5)
        self.relu = nn.ReLU()
    def forward(self, x_c, x_d):
        x = torch.cat((x_c, x_d), dim = 0)
        x = self.avg_pool(x).view(x.size()[0], -1)
        x = self.grl(x)
        x = self.relu(self.hidden_layer(x))
        x = self.out(x)
        return x
    
resnet50 = models.resnet50(pretrained=True)
resnet501 = models.resnet50(pretrained=True)
cnn_c = CNN(resnet50).to(device)
cnn_d = CNN(resnet501).to(device)
concate_net = Concate().to(device)
discriminator = Discriminator().to(device)# 判别分布

# Attention based reconstruction

In [12]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.01)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 0.0, 0.01)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.01)
class ReconstructionNet(nn.Module):
    def __init__(self, in_feature, output_size):
        super(ReconstructionNet, self).__init__()
        self.output_size = output_size
        self.up1 = nn.Sequential(
                               nn.Upsample(scale_factor=4, mode='bicubic', align_corners=True),
                               nn.Conv2d(in_feature, 128, 3, bias=False),
                               nn.BatchNorm2d(128),
                               nn.LeakyReLU(0.2),
        )
        self.up2 = nn.Sequential(
                               nn.Upsample(scale_factor=4, mode='bicubic', align_corners=True),
                               nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1, bias=False),
                               nn.BatchNorm2d(64),
                               nn.LeakyReLU(0.2),

        )
        self.up3 = nn.Sequential(
                               nn.Upsample(scale_factor=2, mode='bicubic', align_corners=True),
                               nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1, bias=False),
                               nn.BatchNorm2d(32),
                               nn.LeakyReLU(0.2),
        )
        self.Sigmoid = nn.Sigmoid()#nn.Tanh()
        self.relu = nn.ReLU()
        self.final_conv =nn.Conv2d(32, 3, 3, padding=1) # , padding=1
        # self.final_conv1 =nn.Conv2d(32, 3, kernel_size = 1) # , padding=1
        self.seg_layers = nn.Sequential(self.up1, self.up2, self.up3)


    def forward(self, x):
        x = self.seg_layers(x)
        x = nn.functional.interpolate(x, size=self.output_size, mode='bicubic', align_corners=True)
        x = self.final_conv(x)
        # x = self.final_conv1(x)
        y = self.Sigmoid(x)
        # y = (torch.sin(x) + 1)/2.
        return y

    def get_parameters(self):
        parameter_list = [{"params":self.parameters(), "lr_mult":1, 'decay_mult':2}]
        return parameter_list

'''def reconstruction_loss(pred=None, ground_truth=None, mask=None, crit=None):
    pred1 = pred.view(pred.size()[0], -1)
    ground_truth1 = pred.view(ground_truth.size()[0], -1)
    
    loss = crit(pred.view(pred.size()[0], -1),  ground_truth.view(ground_truth.size()[0], -1))
    return loss '''
def reconstruction_loss(pred=None, ground_truth=None, mask=None, crit=None):
    '''import pdb;
    pdb.set_trace()'''
    pred1 = pred.view(pred.size()[0], -1)
    ground_truth1 = pred.view(ground_truth.size()[0], -1)
    
    loss = crit(pred.view(pred.size()[0], -1),  ground_truth.view(ground_truth.size()[0], -1))
    if mask != None:
        mask1 = torch.cat((mask, mask, mask), 1)
        mask1 = mask1.view(pred.size()[0], -1).detach()
        # weighted loss 
        loss = loss * torch.exp(mask1) # torch.exp()
    loss = torch.mean(torch.sum(loss, dim=1) / loss.size()[1]) # sum or mean
    return loss 
def show_reconstruction_batch(batch_img, mask = False): # show orignal images, attention maps and reconstruction images in one batch
    if mask == False:
        grid_img = torchvision.utils.make_grid(batch_img, nrow=4)
        plt.imshow(grid_img.permute(1, 2, 0).squeeze())
        plt.show()
    elif mask == True:
        # import pdb;pdb.set_trace() # torch.Size([13, 1, 299, 299])
        grid_img = torchvision.utils.make_grid(batch_img, nrow=4)
        plt.imshow(grid_img.permute(1, 2, 0).squeeze())
        plt.show()
    
reconstruct_net_c = ReconstructionNet(in_feature=2048 * 2, output_size=(299, 299)).to(device)
reconstruct_net_d = ReconstructionNet(in_feature=2048 * 2, output_size=(299, 299)).to(device)

In [13]:
learning_rate = 1e-5
learning_rate_re = 1e-5
criterion = nn.CrossEntropyLoss()
opt_list = chain(cnn_c.parameters(), cnn_d.parameters(), concate_net.parameters(), reconstruct_net_c.parameters(), reconstruct_net_d.parameters(), discriminator.parameters())
# optimizer = optim.Adam(chain(reconstruct_net_c.parameters(), reconstruct_net_d.parameters(), concate_net.parameters(), cnn_c.parameters(), cnn_d.parameters()), lr=learning_rate , weight_decay=0.0001) #
optimizer = optim.Adam(opt_list, lr=learning_rate, weight_decay=0.0001) # , weight_decay=0.0001
# optimizer_con = optim.Adam(chain(concate_net.parameters(), discriminator.parameters()), lr=learning_rate) # , weight_decay=0.0001
# optimizer_re = optim.Adam(chain(reconstruct_net_c.parameters(), reconstruct_net_d.parameters()), lr=learning_rate_re) # , weight_decay=0.0001
# criterion_recon = nn.MSELoss()
criterion_recon = nn.MSELoss(reduction='none')
criterion_l1 = nn.L1Loss(reduction='none')
def generate_label(batch_size):
    l_c = torch.zeros(batch_size)
    l_d = torch.ones(batch_size)
    label = torch.cat((l_c, l_d), dim=0)
    return label

def test_fun(dataloader, model_c, model_d, model_concate, model_recon_c, model_recon_d, epoch):
    model_c.eval()
    model_d.eval()
    model_concate.eval()
    loss_diag_count = []
    loss_crit_pn_count = []
    loss_crit_str_count = []
    loss_crit_pig_count = []
    loss_crit_rs_count = []
    loss_crit_dag_count = []
    loss_crit_bwv_count = []
    loss_crit_vs_count = []
    label_true = {'diag':[], 'pn':[], 'str':[], 'pig':[], 'rs':[], 'dag':[], 'bwv':[], 'vs':[]}
    pred_all = {'diag':[], 'pn':[], 'str':[], 'pig':[], 'rs':[], 'dag':[], 'bwv':[], 'vs':[]}
    with torch.no_grad():
        for index, data in enumerate(dataloader):
            img_c, img_d, lab_diag, lab_crit = data
            label_true['diag'].extend(lab_diag)
            img_c, img_d, lab_diag = img_c.to(device), img_d.to(device), lab_diag.to(device)
            # print(img_c.shape)
            # import pdb;pdb.set_trace()
            label_true['pn'].extend(lab_crit[:, 0]), \
                        label_true['str'].extend(lab_crit[:, 1]), label_true['pig'].extend(lab_crit[:, 2]), label_true['rs'].extend(lab_crit[:, 3]), \
                        label_true['dag'].extend(lab_crit[:, 4]), label_true['bwv'].extend(lab_crit[:, 5]), label_true['vs'].extend(lab_crit[:, 6])
            lab_crit_pn, lab_crit_str, lab_crit_pig, lab_crit_rs, lab_crit_dag, lab_crit_bwv, lab_crit_vs = lab_crit[:, 0].to(device), lab_crit[:, 1].to(device), lab_crit[:, 2].to(device), lab_crit[:, 3].to(device), lab_crit[:, 4].to(device), lab_crit[:, 5].to(device), lab_crit[:, 6].to(device)

            # print(lab_crit_dag)
            feature_c = model_c(img_c)# predict for each class
            feature_d = model_d(img_d)# predict for each class
            prediction = model_concate(feature_c, feature_d)
            out_diag, out_crit_pn, out_crit_str, out_crit_pig, out_crit_rs, out_crit_dag, out_crit_bwv, out_crit_vs, \
                        att_mask_c, att_mask_d, att_feature = prediction
            
            recon_pred_c = model_recon_c(att_feature)
            recon_pred_d = model_recon_d(att_feature)

            pred_all['diag'].extend(out_diag.cpu().detach().numpy()), pred_all['pn'].extend(out_crit_pn.cpu().detach().numpy()), \
                            pred_all['str'].extend(out_crit_str.cpu().detach().numpy()), pred_all['pig'].extend(out_crit_pig.cpu().detach().numpy()), pred_all['rs'].extend(out_crit_rs.cpu().detach().numpy()), \
                            pred_all['dag'].extend(out_crit_dag.cpu().detach().numpy()), pred_all['bwv'].extend(out_crit_bwv.cpu().detach().numpy()), pred_all['vs'].extend(out_crit_vs.cpu().detach().numpy())
            loss_diag = criterion(out_diag, lab_diag)
            loss_crit_pn = criterion(out_crit_pn, lab_crit_pn)
            loss_crit_str = criterion(out_crit_str, lab_crit_str)
            loss_crit_pig = criterion(out_crit_pig, lab_crit_pig)
            loss_crit_rs = criterion(out_crit_rs, lab_crit_rs)
            loss_crit_dag = criterion(out_crit_dag, lab_crit_dag)
            loss_crit_bwv = criterion(out_crit_bwv, lab_crit_bwv)
            loss_crit_vs = criterion(out_crit_vs, lab_crit_vs)
            loss_diag_count.append(loss_diag.item())
            loss_crit_pn_count.append(loss_crit_pn.item())
            loss_crit_str_count.append(loss_crit_str.item())
            loss_crit_pig_count.append(loss_crit_pig.item())
            loss_crit_rs_count.append(loss_crit_rs.item())
            loss_crit_dag_count.append(loss_crit_dag.item())
            loss_crit_bwv_count.append(loss_crit_bwv.item())
            loss_crit_vs_count.append(loss_crit_vs.item())
        print("Epoch: {} test loss, Diag loss: {:.4f}, PN loss: {:.4f}, STR loss: {:.4f}, PIG loss: {:.4f}, RS loss: {:.4f}, DaG loss: {:.4f}, BWV loss: {:.4f}, VS loss: {:.4f}".format(
                        epoch, np.average(loss_diag_count), np.average(loss_crit_pn_count), np.average(loss_crit_str_count), np.average(loss_crit_pig_count), np.average(loss_crit_rs_count), np.average(loss_crit_dag_count), np.average(loss_crit_bwv_count), np.average(loss_crit_vs_count)))
    return pred_all, label_true

In [14]:
def get_average_acc(acc):
    accs = []
    for key in acc.keys():
        accs.append(acc[key])
    avg_acc = np.average(accs)
    return avg_acc
def get_average_auc(auc):
    aucs = []
    for key in auc.keys():
        if key == 'diag':
            continue
        else:
            for key_i in auc[key].keys():
                if key_i == 'micro' or key_i == 'macro':
                    continue
                else:
                    aucs.append(auc[key][key_i])
    # print(len(aucs))
    avg_auc = np.average(aucs)
    return avg_auc
def get_specificity(pre, true):
    sen = {}
    for key in pre:
        sen[key] = specificity_score(np.array(pre[key]).argmax(axis=1), true[key], average=None)
    return sen
def get_confusion_matrix(pre, true): # recall and precision
    confusion_metric = {}
    for key in pre:
        # import pdb;pdb.set_trace()
        confusion_metric[key] = classification_report(np.array(pre[key]).argmax(axis=1), true[key], zero_division  = 1, output_dict=True)
    return confusion_metric

In [17]:
def train_fun(dataloader, model_c, model_d, model_concate, model_recon_c, model_recon_d, optimizer, epoch):
    model_c.train()
    model_d.train()
    model_concate.train()
    loss_diag_count = []
    loss_crit_pn_count = []
    loss_crit_str_count = []
    loss_crit_pig_count = []
    loss_crit_rs_count = []
    loss_crit_dag_count = []
    loss_crit_bwv_count = []
    loss_crit_vs_count = []
    label_true = {'diag':[], 'pn':[], 'str':[], 'pig':[], 'rs':[], 'dag':[], 'bwv':[], 'vs':[]}
    pred_all = {'diag':[], 'pn':[], 'str':[], 'pig':[], 'rs':[], 'dag':[], 'bwv':[], 'vs':[]}
    for index, data in enumerate(dataloader):
        img_c, img_d, lab_diag, lab_crit = data
        label_true['diag'].extend(lab_diag)
        img_c, img_d, lab_diag = img_c.to(device), img_d.to(device), lab_diag.to(device)
        # print(img_c.shape)
        # import pdb;pdb.set_trace()
        label_true['pn'].extend(lab_crit[:, 0]), \
                    label_true['str'].extend(lab_crit[:, 1]), label_true['pig'].extend(lab_crit[:, 2]), label_true['rs'].extend(lab_crit[:, 3]), \
                    label_true['dag'].extend(lab_crit[:, 4]), label_true['bwv'].extend(lab_crit[:, 5]), label_true['vs'].extend(lab_crit[:, 6])
        lab_crit_pn, lab_crit_str, lab_crit_pig, lab_crit_rs, lab_crit_dag, lab_crit_bwv, lab_crit_vs = lab_crit[:, 0].to(device), lab_crit[:, 1].to(device), lab_crit[:, 2].to(device), lab_crit[:, 3].to(device), lab_crit[:, 4].to(device), lab_crit[:, 5].to(device), lab_crit[:, 6].to(device)

        optimizer.zero_grad()

        # print(lab_crit_dag)
        feature_c = model_c(img_c)# predict for each class
        feature_d = model_d(img_d)# predict for each class
        prediction = model_concate(feature_c, feature_d)
        out_diag, out_crit_pn, out_crit_str, out_crit_pig, out_crit_rs, out_crit_dag, out_crit_bwv, out_crit_vs, \
                    att_mask_c, att_mask_d, att_feature = prediction
        
        recon_pred_c = model_recon_c(att_feature)
        recon_pred_d = model_recon_d(att_feature)

        pred_all['diag'].extend(out_diag.cpu().detach().numpy()), pred_all['pn'].extend(out_crit_pn.cpu().detach().numpy()), \
                        pred_all['str'].extend(out_crit_str.cpu().detach().numpy()), pred_all['pig'].extend(out_crit_pig.cpu().detach().numpy()), pred_all['rs'].extend(out_crit_rs.cpu().detach().numpy()), \
                        pred_all['dag'].extend(out_crit_dag.cpu().detach().numpy()), pred_all['bwv'].extend(out_crit_bwv.cpu().detach().numpy()), pred_all['vs'].extend(out_crit_vs.cpu().detach().numpy())
        loss_diag = criterion(out_diag, lab_diag)
        loss_crit_pn = criterion(out_crit_pn, lab_crit_pn)
        loss_crit_str = criterion(out_crit_str, lab_crit_str)
        loss_crit_pig = criterion(out_crit_pig, lab_crit_pig)
        loss_crit_rs = criterion(out_crit_rs, lab_crit_rs)
        loss_crit_dag = criterion(out_crit_dag, lab_crit_dag)
        loss_crit_bwv = criterion(out_crit_bwv, lab_crit_bwv)
        loss_crit_vs = criterion(out_crit_vs, lab_crit_vs)
        loss_diag_count.append(loss_diag.item())
        loss_crit_pn_count.append(loss_crit_pn.item())
        loss_crit_str_count.append(loss_crit_str.item())
        loss_crit_pig_count.append(loss_crit_pig.item())
        loss_crit_rs_count.append(loss_crit_rs.item())
        loss_crit_dag_count.append(loss_crit_dag.item())
        loss_crit_bwv_count.append(loss_crit_bwv.item())
        loss_crit_vs_count.append(loss_crit_vs.item())

        loss = loss_diag + loss_crit_pn + loss_crit_str + loss_crit_pig + loss_crit_rs + loss_crit_dag + loss_crit_bwv + loss_crit_vs
        loss.backward()
        optimizer.step()

    print("Epoch: {} train loss, Diag loss: {:.4f}, PN loss: {:.4f}, STR loss: {:.4f}, PIG loss: {:.4f}, RS loss: {:.4f}, DaG loss: {:.4f}, BWV loss: {:.4f}, VS loss: {:.4f}".format(
                    epoch, np.average(loss_diag_count), np.average(loss_crit_pn_count), np.average(loss_crit_str_count), np.average(loss_crit_pig_count), np.average(loss_crit_rs_count), np.average(loss_crit_dag_count), np.average(loss_crit_bwv_count), np.average(loss_crit_vs_count)))
    return pred_all, label_true


trainloader = torch.utils.data.DataLoader(test_dataset, batch_size=12, shuffle=False, num_workers=4)


from time import time
total_time_train = 0
for epoch in range(5):
        # training
        start_time_epoch_train = time()

        print("Epoch {} begin train...".format(epoch))
        pred_all_train, label_true_train = train_fun(trainloader, cnn_c, cnn_d, concate_net, reconstruct_net_c, reconstruct_net_d, optimizer, epoch)
        auc_all_train, acc_all_train, con_all_train = metric(pred_all_train, label_true_train, show=False)
        avg_acc = get_average_acc(acc_all_train)# get the average acc
        avg_auc = get_average_auc(auc_all_train)
        con_metric = get_confusion_matrix(pred_all_train, label_true_train) # compute recall and precision
        specificity = get_specificity(pred_all_train, label_true_train)
        # sens, spec, prec = get_confusion_matrix(con_all_test)
        print(avg_acc, avg_auc)
        # if i % 10 == 0 or i == (epochs - 1):
        if (record_acc+record_auc) <= (avg_acc + avg_auc):
            record_acc = avg_acc
            record_auc = avg_auc
            print("Train metics on epoch {}:".format(epoch))
            print(auc_all_train)
            print(acc_all_train)
            print("Current best average ACC: {:.4f}".format(avg_acc))
            print("Current average AUC: {:.4f}".format(avg_auc))
            print()

        end_time_epoch_train = time()
        epoch_time_train = end_time_epoch_train - start_time_epoch_train
        total_time_train += epoch_time_train

Epoch 0 begin train...


Epoch: 0 train loss, Diag loss: 1.2580, PN loss: 0.6311, STR loss: 0.5189, PIG loss: 0.4011, RS loss: 0.2221, DaG loss: 0.6555, BWV loss: 0.2774, VS loss: 0.3361
0.8132911392405063 0.9372441951816893
Test metics on epoch 1:
{'diag': {0: 0.6368733509234829, 1: 0.5922322540473226, 2: 0.5901192160032329, 3: 0.6863380281690141, 4: 0.5431131019036954, 'micro': 0.8056369171607114, 'macro': 0.6114560493097589}, 'pn': {0: 0.9071183349426029, 1: 0.9264455080596358, 2: 0.9157231360820337, 'micro': 0.9149270950168242, 'macro': 0.9182676287829574}, 'str': {0: 0.9149326115152541, 1: 0.8748381248381248, 2: 0.9193822011733936, 'micro': 0.9319179618650857, 'macro': 0.9058376981152738}, 'pig': {0: 0.9778131191990823, 1: 0.9852305475504323, 2: 0.970479704797048, 'micro': 0.9741547828873578, 'macro': 0.9789697450152546}, 'rs': {0: 0.9865508911666775, 1: 0.9865508911666775, 'micro': 0.9861688831917963, 'macro': 0.9875009683070903}, 'dag': {0: 0.8962033898305084, 1: 0.9204246466377042, 2: 0.903643808635256

In [ ]:
fa

In [15]:
epochs = 2000
record_acc = 0.
record_auc = 0.

model_name_c = './checkpoint/feature_extraction_c_fusion_9-12_21.pth'# './checkpoint/feature_extraction_concate_discrinimator_0713_c1_two_stream.pth' # 3 ahieved the best performance 
model_name_d = './checkpoint/feature_extraction_d_fusion_9-12_21.pth' #'./checkpoint/feature_extraction_concate_discrinimator_0713_d1_two_stream.pth'
model_name_concate = './checkpoint/concatenate_fusion_9-12_21.pth'# './checkpoint/concate_discrinimator_0713_concatenate1_two_stream.pth'
model_name_dis_c = './checkpoint/discriminator_fusion_9-12_21.pth'# './checkpoint/reconstruct_concate_discrinimator_0713_c1_two_stream.pth'
model_name_recon_c = './checkpoint/reconstruct_c_fusion_9-12_21.pth'# './checkpoint/feature_extraction_concate_recon_0713_c1_two_stream.pth' # 3 ahieved the best performance
model_name_recon_d = './checkpoint/reconstruct_c_fusion_9-12_21.pth'# './checkpoint/feature_extraction_concate_recon_0713_d1_two_stream.pth'

checkpoint_c = torch.load(model_name_c)
cnn_c.load_state_dict(checkpoint_c)
checkpoint_d = torch.load(model_name_d)
cnn_d.load_state_dict(checkpoint_d)
checkpoint_concate_net = torch.load(model_name_concate)
concate_net.load_state_dict(checkpoint_concate_net)

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=48,
                                              shuffle=False, num_workers=4)
i=1
print("Epoch {} begin testing...".format(i))
pred_all_test, label_true_test = test_fun(testloader, cnn_c, cnn_d, concate_net, reconstruct_net_c, reconstruct_net_d, i)
auc_all_test, acc_all_test, con_all_test = metric(pred_all_test, label_true_test, show=False)
avg_acc = get_average_acc(acc_all_test)# get the average acc
avg_auc = get_average_auc(auc_all_test)
con_metric = get_confusion_matrix(pred_all_test, label_true_test) # compute recall and precision
specificity = get_specificity(pred_all_test, label_true_test)
# sens, spec, prec = get_confusion_matrix(con_all_test)
print(avg_acc, avg_auc)
# if i % 10 == 0 or i == (epochs - 1):
if (record_acc+record_auc) <= (avg_acc + avg_auc):
    record_acc = avg_acc
    record_auc = avg_auc
    print("Test metics on epoch {}:".format(i))
    print(auc_all_test)
    print(acc_all_test)
    print("Current best average ACC: {:.4f}".format(avg_acc))
    print("Current average AUC: {:.4f}".format(avg_auc))
    # print('confusion_matrix' + str(con_metric))


Epoch 1 begin testing...
Epoch: 1 test loss, Diag loss: 0.8220, PN loss: 0.6805, STR loss: 0.5907, PIG loss: 0.7437, RS loss: 0.3653, DaG loss: 0.8746, BWV loss: 0.2866, VS loss: 0.4405
0.7591772151898735 0.8473569795321536
Test metics on epoch 1:
{'diag': {0: 0.941787598944591, 1: 0.8965338314653383, 2: 0.8903482184953189, 3: 0.9059859154929577, 4: 0.8171892497200448, 'micro': 0.9339593013940073, 'macro': 0.8917050790802882}, 'pn': {0: 0.892018023817187, 1: 0.844886394894647, 2: 0.8205155593534144, 'micro': 0.8611728889601025, 'macro': 0.8542553682972424}, 'str': {0: 0.8300343991428409, 1: 0.8954286454286454, 2: 0.8073443132819679, 'micro': 0.8978497035731454, 'macro': 0.846272765996983}, 'pig': {0: 0.8073573886745229, 1: 0.8515249759846302, 2: 0.8338590643970956, 'micro': 0.8701810607274474, 'macro': 0.8326071577107976}, 'rs': {0: 0.8675980936214664, 1: 0.8675980936214662, 'micro': 0.906771350745073, 'macro': 0.8702723924781556}, 'dag': {0: 0.7717966101694915, 1: 0.7768463562381448, 